In [11]:
import requests
import time
import pandas as pd
from api_keys import api_key

In [12]:
# Load the CSV file
file_path = 'Resourses/Fortune 500 Data.csv'  # Update this with the correct path to your CSV file
df = pd.read_csv(file_path)

# Assuming the tickers are in the second column (index 1)
tickers = df.iloc[:, 1].unique()  # Extract unique tickers to avoid duplicate API calls

# Limit to the first 100 tickers
tickers_100 = tickers[100:200]
tickers_100

array(['ARW', 'OXY', 'TRV', 'NOC', 'HON', 'MMM', 'USFD', 'WBD', 'LEN',
       'DHI', 'JBL', 'LNG', 'AVGO', 'KMX', 'SBUX', 'MOH', 'UBER', 'PM',
       'NFLX', 'NRG', 'MDLZ', 'DHR', 'CRM', 'PARA', 'CBRE', 'MU', 'V',
       'SO', 'UNFI', 'PCAR', 'DUK', 'LLY', 'HPE', 'DLTR', 'LAD', 'CMI',
       'PAG', 'PYPL', 'USB', 'GILD', 'AN', 'NVDA', 'KHC', 'AMGN', 'AMAT',
       'EOG', 'TFC', 'M', 'UNP', 'CHRW', 'RAD', 'CEG', 'AVT', 'PXD',
       'LUV', 'CDW', 'AMD', 'PNC', 'MCD', 'CLF', 'FCX', 'BLDR', 'OKE',
       'HIG', 'SCHW', 'STLD', 'MA', 'SHW', 'GPC', 'PCG', 'WCC', 'MUSA',
       'WRK', 'IP', 'BKR', 'X', 'NEE', 'TRGP', 'LEA', 'JLL', 'GT', 'MAR',
       'MMC', 'MO', 'CPNG', 'CARR', 'HAL', 'DK', 'KMB', 'TXN', 'BK',
       'MAN', 'WHR', 'WM', 'AEP', 'EMR', 'AFL', 'CTSH', 'BDX', 'BJ'],
      dtype=object)

In [13]:
# Manually add January 3, 2023
business_days_2023 = ['2023-01-03']

# Generate the first business day for each month from February to December 2023
additional_days = pd.date_range(start='2023-02-01', end='2023-12-31', freq='BMS')
additional_days = additional_days.strftime('%Y-%m-%d').tolist()

# Combine the lists
business_days_2023.extend(additional_days)

# Display the generated dates
print(business_days_2023)


['2023-01-03', '2023-02-01', '2023-03-01', '2023-04-03', '2023-05-01', '2023-06-01', '2023-07-03', '2023-08-01', '2023-09-01', '2023-10-02', '2023-11-01', '2023-12-01']


In [14]:
# Assuming tickers and business_days_2023 are already defined in your script

# Limit to the first 5 tickers
#first_100_tickers = tickers[:5]

# Limit to the first date
#business_days_2023 = business_days_2023[:2]
#business_days_2023

In [15]:
# Function to make the API call
def make_api_call(ticker, date, api_key):
    url = f"https://api.polygon.io/v1/open-close/{ticker}/{date}?adjusted=true&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Exclude 'status' and rename 'from' and 'symbol'
        return {
            "date": data["from"],
            "ticker": data["symbol"],
            "open": data["open"],
            "high": data["high"],
            "low": data["low"],
            "close": data["close"],
            "volume": data["volume"],
            "afterHours": data["afterHours"],
            "preMarket": data["preMarket"]
        }
    else:
        return None

def fetch_data_for_tickers(tickers, dates, api_key):
    # Initialize an empty DataFrame inside the function
    columns = ["date", "ticker", "open", "high", "low", "close", "volume", "afterHours", "preMarket"]
    df = pd.DataFrame(columns=columns)

    for ticker in tickers:
        for date in dates:
            result = make_api_call(ticker, date, api_key)
            if result:
                # Append the result to the DataFrame
                df = pd.concat([df, pd.DataFrame([result])], ignore_index=True)
            time.sleep(12)  # Sleep to limit to 5 calls per minute
    return df

# Call the function and store the results in the DataFrame
tickers_100 = fetch_data_for_tickers(tickers_100, business_days_2023, api_key)


In [16]:
data = pd.read_csv("Resourses/stockdata.csv")
data

,Index,date,ticker,open,high,low,close,volume,afterHours,preMarket
0,0,2023-01-03,WMT,142.55,143.71,142.0900,143.60,5749895.0,143.60,142.04
1,1,2023-02-01,WMT,143.66,145.61,142.7000,144.67,4283059.0,144.55,143.05
2,2,2023-03-01,WMT,141.06,141.28,139.3701,140.15,4726732.0,140.02,142.16
3,3,2023-04-03,WMT,147.73,148.95,147.5000,148.69,6588882.0,148.57,147.44
4,4,2023-05-01,WMT,151.04,152.54,150.9300,151.59,3803383.0,151.59,150.99
...,...,...,...,...,...,...,...,...,...,...
1173,1173,2023-08-01,DG,169.33,169.93,167.8900,168.44,1179146.0,168.44,168.55
1174,1174,2023-09-01,DG,136.55,136.59,128.5800,130.27,10694875.0,130.27,136.75
1175,1175,2023-10-02,DG,105.22,105.54,102.4700,103.78,4503524.0,103.85,106.50
1176,1176,2023-11-01,DG,119.15,119.82,116.0800,116.36,3048305.0,116.37,118.50


In [17]:
combined_data = pd.concat([data, tickers_100], ignore_index=True)

In [18]:
combined_data.to_csv("Resourses/stockdata.csv", index_label="Index")